In [ ]:
import torch
import torch.nn as nn
from transformers import (BertTokenizer, BertModel, BertConfig,
                          BertForSequenceClassification,
                          PretrainedConfig, PreTrainedModel,
                          TrainingArguments, TrainerCallback, Trainer,
                          DataCollatorWithPadding)
from datasets import Dataset, DatasetDict, load_dataset
import pandas # for proper formatting of datasets

import random

from Synthesizer import Synthesizer, resize_embedding, compute_accuracy

In [ ]:
class StopCallback(TrainerCallback):
    def on_epoch_begin(self, args, state, control, logs=None, **kwargs):
        if state.best_metric == 1.0:
            control.should_training_stop = True

In [ ]:
# globals
maximum_program_length = 256
maximum_data_length = 256
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
vocab_size = len(tokenizer)
numeric_tokens = [tokenizer(f'{i}')["input_ids"][1] for i in range(maximum_program_length)]
[print(tokenizer.decode([numeric_tokens[i]]), end = " ") for i in range(maximum_program_length)];

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 

# Experiment 1

In [ ]:
def generate_routines(n_subroutines, subroutine_size=10):
    # create a predefined set of subroutines, ie. a mapping of (token -> list of instructions of form (A, B, C))
    return {i: [( random.randrange(0, maximum_data_length),
                  random.randrange(0, maximum_data_length),
                  random.randrange(0, maximum_program_length) ) for _ in range(subroutine_size)]
            for i in range(vocab_size, vocab_size + n_subroutines)}

def ex1_data(routines, subset = None):
    if subset is not None:
        routines = dict(list(routines.items())[:subset])
    data = []
    for routine_id, instructions in routines.items():
        for i in range(len(instructions)):
            instruction = instructions[i]
            data.append({"input_ids": [numeric_tokens[i], routine_id],
                         "label_A": [instruction[0]],
                         "label_B": [instruction[1]],
                         "label_C": [instruction[2]] })
    dataset = Dataset.from_dict({"train": data, "validation": data})
    dataset.set_format('pt')
    return dataset

def ex1_run(n_subroutines, random_seed):
    torch.manual_seed(random_seed)
    random.seed(random_seed)


    config = BertConfig(num_attention_heads=4, hidden_size = 256,
                        intermediate_size=1024, num_hidden_layers = 4,
                        postion_embedding_type = "relative_key_query")

    synthesizer = Synthesizer(maximum_program_length, maximum_data_length, config)

    routines = generate_routines(n_subroutines = n_subroutines, subroutine_size = 256)
    dataset = ex1_data(routines)
    resize_embedding(synthesizer, new_size = vocab_size + n_subroutines)
    
    training_args = TrainingArguments(
        output_dir = "./results",
        logging_strategy = "epoch",
        evaluation_strategy = "epoch",
        save_strategy = "epoch",
        report_to = "none",
        learning_rate = 1e-3,
        num_train_epochs = 250,
        per_device_train_batch_size = len(dataset) // 20,
        per_device_eval_batch_size = len(dataset) // 20,
        metric_for_best_model = "accuracy",
        greater_is_better = True,
        seed = random_seed,
    )
    
    trainer = Trainer(
        model=synthesizer,
        args=training_args,
        train_dataset=dataset["train"],
        eval_dataset=dataset["validation"],
        data_collator=data_collator,
        compute_metrics=compute_accuracy,
        callbacks = [StopCallback],
    )

    print(f"running subroutine size {n_subroutines}, with random seed {random_seed}, batch_size is {len(dataset) // 20}")
    trainer.train()

    assert trainer.state.best_metric == 1.0
    
    return trainer.state.global_step


def rerun_ex1(sizes, random_seeds):
    results = {s: [] for s in sizes}
    for size in sizes:
        for seed in random_seeds:
            steps = ex1_run(size, seed)
            results[size].append(steps)
    return results

# learning rates tried 1e-4, 1e-3, 5e-3
# batch size came from preliminary experiment where we found larger batch sizes helped with memorization
# the divided by 20 was arbitrary

In [ ]:
import logging
logging.disable(logging.INFO)

n_subroutines = [16, 32, 64, 128]
seeds_to_use = [1, 2, 3, 4, 5]
ex1_results = rerun_ex1(n_subroutines, seeds_to_use)

In [ ]:
ex1_df = pandas.DataFrame(ex1_results)
ex1_df.to_csv("ex1_results.csv", index = False)

#ex1_df = pandas.read_csv("ex1_results.csv")
display(ex1_df.mean())
display(ex1_df.std())

16     1030.0
32     1366.0
64     1907.8
128    2991.4
dtype: float64

16      74.246212
32      95.081544
64     153.170493
128     72.137369
dtype: float64

# Experiment (2-3)

In [ ]:
# Experiment 2 baseline sst2 training
def preprocess(examples):
    return tokenizer(examples["sentence"], truncation=True)

# Experiment 2 preprocessing for soft program
# program description to append to input (get rid of cls token though)
ex2_program_description = tokenizer("Classify this input")["input_ids"][1:]
cls_token_id = tokenizer.cls_token_id
def synthetic_preprocess(examples):
    result = tokenizer(examples["sentence"], truncation=True,
                       max_length = tokenizer.model_max_length - len(ex2_program_description) - 1)
    
    # add the extra dimensionality (also put cls token in front)
    input_ids = result["input_ids"][1:]
    input_ids = [cls_token_id] + [numeric_tokens[0]] + ex2_program_description + input_ids
    result["input_ids"] = input_ids
    
    if examples["label"] == 1:
        result["label_A"] = [0]
        result["label_B"] = [2]
        result["label_C"] = [0]
    else:
        result["label_A"] = [1]
        result["label_B"] = [2]
        result["label_C"] = [0]
    return result

In [ ]:
from datasets import load_metric
 
def compute_metrics(eval_pred):
    load_accuracy = load_metric("accuracy")

    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
    return {"accuracy": accuracy}

def compute_sst2_accuracy(eval_prediction):
    label_A, label_B, label_C = eval_prediction.predictions[0]
    logits_A, logits_B, logits_C = eval_prediction.predictions[1]
    
    acc_A = (logits_A.argmax(axis = 1) == label_A.reshape(-1)).mean()
    acc_B = (logits_B.argmax(axis = 1) == label_B.reshape(-1)).mean()
    acc_C = (logits_C.argmax(axis = 1) == label_C.reshape(-1)).mean()

    acc = (acc_A + acc_B + acc_C) / 3
    return {'accuracy': acc, 'accuracy_A': acc_A, 'accuracy_B': acc_B, 'accuracy_C': acc_C}

In [ ]:
# experiment 3 dataset generation/preprocessing
def generate_routines(n_subroutines, subroutine_size=10):
    # create a predefined set of subroutines, ie. a mapping of (token -> list of instructions of form (A, B, C))
    return {i: [( random.randrange(1, maximum_data_length),
                  random.randrange(1, maximum_data_length),
                  random.randrange(1, maximum_program_length) ) for _ in range(subroutine_size)]
            for i in range(vocab_size, vocab_size + n_subroutines)}

def ex3_programs(routines, subset = None):
    if subset is not None:
        routines = dict(list(routines.items())[:subset])
    data = []
    for routine_id, instructions in routines.items():
        for i in range(len(instructions)):
            instruction = instructions[i]
            data.append({"instruction_num": [numeric_tokens[i]],
                         "routine_id": [routine_id],
                         "label_A": [instruction[0]],
                         "label_B": [instruction[1]],
                         "label_C": [instruction[2]] })
    return pandas.DataFrame(data)

# tokenize but get rid of cls and sep token (they will be added back later)
ex3_program_description = tokenizer("As long as the input text is postive, run ")["input_ids"][1:-1]
def ex3_preprocess(examples):
    # tokenize making room for instruction number, routine id and a seperator token
    result = tokenizer(examples["sentence"], truncation=True,
                       max_length = tokenizer.model_max_length - len(ex3_program_description) - 3)
    
    # remove cls token since it should go in front
    review_ids = result["input_ids"][1:]
    
    # randomly sample from programs table
    global progams
    program_sample = programs.sample(n = 1)
    
    # form task input ex.
    # "[CLS] [instruction number] As long as the input text is positive run [program token] [SEP] this movie was terrible [SEP]"
    result["input_ids"] = (
        [tokenizer.cls_token_id]
        + program_sample["instruction_num"].item()
        + ex3_program_description
        + program_sample["routine_id"].item()
        + [tokenizer.sep_token_id]
        + review_ids # sep token is already there
    )
    
    if examples["label"] == 1:
        # positive sentiment, program should run normal instruction
        label_A = program_sample["label_A"].item()
        label_B = program_sample["label_B"].item()
        label_C = program_sample["label_C"].item()
    else:
        # negative sentiment so the program should immediately halt
        label_A = [0]
        label_B = [0]
        label_C = [0]
        
    result["label_A"] = label_A
    result["label_B"] = label_B
    result["label_C"] = label_C
    
    return result

In [ ]:
import logging
logging.disable(logging.INFO)


def run_ex2_3(random_seed):
    torch.manual_seed(random_seed)
    random.seed(random_seed)
    
    
    # get all datasets
    sst = load_dataset('sst2')

    dataset = sst.map(preprocess)
    dataset.set_format('pt')
    dataset = dataset.remove_columns(["idx", "sentence", "token_type_ids", "attention_mask"])

    synthetic_dataset = sst.map(synthetic_preprocess)
    synthetic_dataset.set_format('pt')
    synthetic_dataset = synthetic_dataset.remove_columns(["idx", "sentence", "label", "token_type_ids", "attention_mask"])

    num_programs = 10
    routines = generate_routines(num_programs, 10)
    global programs
    programs = ex3_programs(routines)
    ex3_sst = sst.map(ex3_preprocess).remove_columns(["idx", "sentence", "token_type_ids", "attention_mask", "label"])
    ex3_sst.set_format('pt')
    
    training_args = TrainingArguments(
        output_dir = "./results",
        logging_strategy = "steps",
        logging_steps = 64,
        evaluation_strategy = "steps",
        eval_steps = 64,
        save_strategy = "steps",
        save_steps = 64,
        report_to = "none",
        learning_rate = 1e-4,
        num_train_epochs = 2,
        per_device_train_batch_size = 32,
        per_device_eval_batch_size = 64,
        metric_for_best_model = "eval_accuracy",
        greater_is_better = True,
        seed = random_seed,
        load_best_model_at_end = True,
    )
    # learning rates tried 1e-4, 1e-3
    
    
    # Experiment 2 baseline training run
    base_model = BertForSequenceClassification(
        BertConfig(num_attention_heads=4, hidden_size = 256,
        intermediate_size=1024, num_hidden_layers = 4)
    )

    base_trainer = Trainer(
        model=base_model,
        args=training_args,
        train_dataset=dataset["train"],
        eval_dataset=dataset["validation"],
        data_collator=data_collator,
        compute_metrics=compute_metrics
    )
    base_trainer.train()

    
    # Experiment 2 soft program run
    config = BertConfig(num_attention_heads=4, hidden_size = 256,
                        intermediate_size=1024, num_hidden_layers = 4,
                        postion_embedding_type = "relative_key_query")
    synthesizer = Synthesizer(maximum_program_length, maximum_data_length, config)

    training_args.metric_for_best_model = "eval_accuracy_A"
    synthetic_trainer = Trainer(
        model=synthesizer,
        args=training_args,
        train_dataset=synthetic_dataset["train"],
        eval_dataset=synthetic_dataset["validation"],
        data_collator=data_collator,
        compute_metrics=compute_sst2_accuracy
    )
    synthetic_trainer.train()

    
    # Experiment 3 imperative run
    config = BertConfig(num_attention_heads=4, hidden_size = 256,
                        intermediate_size=1024, num_hidden_layers = 4,
                        postion_embedding_type = "relative_key_query")
    synthesizer = Synthesizer(maximum_program_length, maximum_data_length, config)
    resize_embedding(synthesizer, new_size = vocab_size + num_programs)

    training_args.metric_for_best_model = "eval_accuracy"
    imperative_trainer = Trainer(
        model=synthesizer,
        args=training_args,
        train_dataset=ex3_sst["train"],
        eval_dataset=ex3_sst["validation"],
        data_collator=data_collator,
        compute_metrics=compute_sst2_accuracy
    )
    imperative_trainer.train()
    
    return (base_trainer, synthetic_trainer, imperative_trainer)




def ex2_3_results(seeds):
    metrics = []
    for seed in seeds:
        base_trainer, synthetic_trainer, imperative_trainer = run_ex2_3(seed)
        metrics.append(
            {"base_accuracy": base_trainer.state.best_metric,
             "synthetic_accuracy": synthetic_trainer.state.best_metric,
             "imperative_accuracy": imperative_trainer.state.best_metric}
        )
    
    return metrics, bert_results, synthesizer_results, imperative_results

In [ ]:
accuracies, bert_results, synthesizer_results, imperative_results = ex2_3_results([1, 2, 3, 4, 5])

In [ ]:
acc_df = pandas.DataFrame(accuracies)
acc_df.mean(), acc_df.std()

In [ ]:
params = {
   'axes.labelsize': 15,
   'font.size': 15,
   'legend.fontsize': 15,
   'xtick.labelsize': 15,
   'ytick.labelsize': 15,
   'figure.figsize': [8, 6]
   }
matplotlib.rcParams.update(params)

# plot last run (getting rid of last measurement as it is just a repeat)
bert_results = pandas.DataFrame(base_trainer.state.log_history[1::2]).set_index("step")["eval_accuracy"][:-1]
synthesizer_results = pandas.DataFrame(synthetic_trainer.state.log_history[1::2]).set_index("step")["eval_accuracy_A"][:-1]
imperative_results = pandas.DataFrame(imperative_trainer.state.log_history[1::2]).set_index("step")["eval_accuracy"][:-1]
plt.plot(bert_results, label = "BERT for Sequence Classification")
plt.plot(synthesizer_results, label = "Synthesizer (Soft-Instruction Task)")
plt.plot(imperative_results, label = "Synthesizer (Soft-Imperative Task)")

plt.xlabel("Step", size = 17)
plt.ylabel("Validation Accuracy", size = 17)
plt.legend()
plt.grid()
plt.savefig('fig.png', dpi = 300)
plt.show()